Wycina fragmenty obrazów zawierające wykryte przez CRAFT napisy. Fragmenty zapisywane są w folderach nazwanych z użyciem id filmu.

In [31]:
from skimage import io
import numpy as np
from pathlib import Path
import glob

def bounding_box(points):
    x_coordinates, y_coordinates = zip(*points)
    return [(min(x_coordinates), min(y_coordinates)), (max(x_coordinates), max(y_coordinates))]

CRAFT_path_results = Path("./data/CRAFT_result_GB/")
path_images = Path("./data/thumbnailsHQ/GB")

for j, img_path in enumerate(glob.glob(str(path_images) + "/*.png")):
    img_path = Path(img_path)
    img = io.imread(img_path)
    shape = img.shape
    
    bboxes_path = CRAFT_path_results / ('res_' + img_path.stem + ".txt")
    bboxes = []
    with open(bboxes_path, "r") as bboxFile:
        for line in bboxFile.readlines():
            numbers = np.array(line.split(',')).astype(int)
            points = []
            
            for x,y in zip(numbers[::2], numbers[1::2]):
                points.append((x, y))
            bboxes.append(bounding_box(points))

    
    if len(bboxes) > 0:
            Path(CRAFT_path_results / "words" / img_path.stem).mkdir(parents=True, exist_ok=True)
            img_dir = Path(CRAFT_path_results / "words" / img_path.stem)
    
    for i, bbox in enumerate(bboxes):

        img_slice = img[max(bbox[0][1], 0): min(bbox[1][1], shape[0]), max(bbox[0][0], 0):min(bbox[1][0], shape[1])]
        io.imsave(img_dir / f"{i}.png", img_slice)

#     if j == 100:
#         break


<ipython-input-31-e4e3d495332b>:38: UserWarning: data/CRAFT_result_GB/words/c5E9Rns3HOo/1.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)
<ipython-input-31-e4e3d495332b>:38: UserWarning: data/CRAFT_result_GB/words/cIQKKzZtWwQ/0.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)
<ipython-input-31-e4e3d495332b>:38: UserWarning: data/CRAFT_result_GB/words/fzCct3j-pH8/5.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)
<ipython-input-31-e4e3d495332b>:38: UserWarning: data/CRAFT_result_GB/words/ryr75N0nki0/0.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)
<ipython-input-31-e4e3d495332b>:38: UserWarning: data/CRAFT_result_GB/words/UceaB4D0jpo/0.png is a low contrast image
  io.imsave(img_dir / f"{i}.png", img_slice)


Teraz OCR na fragmentach zawierających napisy. Wynikiem jest df z kolumnami [video_id, "word_id", "ocr_text"].

In [9]:
# %pip install -r CRAFT-pytorch/requirements.txt
# %pip install lmdb pillow nltk natsort

  Using cached lmdb-1.1.1-cp38-cp38-manylinux2010_x86_64.whl (306 kB)
  Using cached natsort-7.1.1-py3-none-any.whl (35 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
%run deep-text-recognition-benchmark/demo_modified.py \
--Transformation TPS \
--FeatureExtraction ResNet \
--SequenceModeling BiLSTM \
--Prediction Attn \
--image_folder "data/CRAFT_result_GB/words/" \
--saved_model "deep-text-recognition-benchmark/TPS-ResNet-BiLSTM-Attn.pth" \
--out_df_path "our_data/etap2_ocr_GB.csv"

model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from deep-text-recognition-benchmark/TPS-ResNet-BiLSTM-Attn.pth
